## Preprocessing scraped PFD data

In [25]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI

Below, we can see the result of our web scraping script. It produces 5 fields:
 * The "URL" of the PFD report
 * The "ID", scraped from the judiciary.uk website
 * The "date" of the report. Unhelpfully, this is in a mixture of formats.
 * The "receiver", or who the report was addressed to
 * The "content" of the reports's *concerns* section

In [26]:
data = pd.read_csv('../Data/output.csv')
data.head(n = 5)

,URL,ID,Date,Receiver,Content
0,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0318,Date of report: 13/06/2024,"TO: The Chief Executive, Leicestershire Partne...",During the course of the investigation my inqu...
1,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0311,Date of report: 07/06/2024,TO: 1. NATIONAL AMBULANCE RESILIENCE UNIT (NAR...,Regulation 28 – After Inquest Document Templat...
2,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0298,Date of report: 05/05/2024,"TO: 1. CEO of Quora, 2. The Rt Hon Lucy Fraser...",During the course of the inquest the evidence ...
3,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0297,Date of report: 29/05/2024,"TO: Secretary of State for Justice, 1",During the course of the inquest the evidence ...
4,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0296,Date of report: 03/06/2024,"TO: (1) , Chief Executive, Birmingham and Soli...","During the inquest, the evidence revealed matt..."


Not all reports were scraped successfully, which we can see by *NA* values in the `Receiver` and `Content` fields. Inspecting the URLs of failed reports shows that this is because these reports are actually images, which have been scanned in and uploaded. 

There are alternative Python packages that allow for the reading of text in images, but we'll leave this for now due to time constraints.

In [27]:
# Count number of rows
rows = data.shape[0]

# Count number of rows that have N/A values in "Content" column
na_rows = data['Content'].isna().sum()

# Print with no spaces
print(f"There are", na_rows, "out of", rows, "reports which were unsuccessfully scraped.",
      "This reflects", round(na_rows/rows*100, 2), "% of all reports.",
      "\nThis leaves us with", rows - na_rows, "reports left to analyse.")

# Drop rows with N/A values in "Content" column
data = data.dropna(subset = ['Content'])

There are 155 out of 570 reports which were unsuccessfully scraped. This reflects 27.19 % of all reports. 
This leaves us with 415 reports left to analyse.


### Tokenisation and cleaning

Below we will word-tokenise the data. As additional pre-processing steps, we will also remove stop words and punctuation.

In [38]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Apply word_tokenize to the 'Content' column

data['Tokenized_Content'] = data['Content'].apply(word_tokenize)
data

[nltk_data] Downloading package punkt to /home/sam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,URL,ID,Date,Receiver,Content,Tokenized_Content
0,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0318,Date of report: 13/06/2024,"TO: The Chief Executive, Leicestershire Partne...",During the course of the investigation my inqu...,"[During, the, course, of, the, investigation, ..."
1,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0311,Date of report: 07/06/2024,TO: 1. NATIONAL AMBULANCE RESILIENCE UNIT (NAR...,Regulation 28 – After Inquest Document Templat...,"[Regulation, 28, –, After, Inquest, Document, ..."
2,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0298,Date of report: 05/05/2024,"TO: 1. CEO of Quora, 2. The Rt Hon Lucy Fraser...",During the course of the inquest the evidence ...,"[During, the, course, of, the, inquest, the, e..."
3,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0297,Date of report: 29/05/2024,"TO: Secretary of State for Justice, 1",During the course of the inquest the evidence ...,"[During, the, course, of, the, inquest, the, e..."
4,https://www.judiciary.uk/prevention-of-future-...,Ref: 2024-0296,Date of report: 03/06/2024,"TO: (1) , Chief Executive, Birmingham and Soli...","During the inquest, the evidence revealed matt...","[During, the, inquest, ,, the, evidence, revea..."
...,...,...,...,...,...,...
555,https://www.judiciary.uk/prevention-of-future-...,Ref: 2016-0037,Date of report: 5 February 2016,TO: 1. Dean for Education Barts and the London...,"During the course of the inquest, the evidence...","[During, the, course, of, the, inquest, ,, the..."
559,https://www.judiciary.uk/prevention-of-future-...,Ref: 2015-0465,Date of report: 24 November 2015,"TO: Chief Executive, Lancashire Care NHS Found...",During the course of the inquest the evidence ...,"[During, the, course, of, the, inquest, the, e..."
562,https://www.judiciary.uk/prevention-of-future-...,Ref: 2015-0173,Date of report: 29 April 2015,TO: 1. Ms Wendy Wallace Chief Executive Camden...,"During the course of the inquest, the evidence...","[During, the, course, of, the, inquest, ,, the..."
564,https://www.judiciary.uk/prevention-of-future-...,Ref: 2015-0116,Date of report: 24 March 2015,"TO: National Offender Management Service, Cliv...",During the course of the inquest the evidence ...,"[During, the, course, of, the, inquest, the, e..."


### Removing intro text

Notice in the `Content` field of the scraped reports that all text begins with some intro text. This text differs ever so slightly between reports, but it usually around the lines of:

> *During the course of the investigation my inquiries revealed matters giving rise to concern. In my opinion there is a risk that future deaths could occur unless action is taken. In the circumstances it is my statutory duty to report to you. The MATTERS OF CONCERN are as follows:*

Because of slight grammatical differences between these texts across documents, we can't use regular expressions (regex) to filter these out. Instead, we have to call the OpenAI API to remove this text for us.

Note that for the below code to run on your machine, it requires you to set an environmental variable containing your OpenAI API Key (which I've called `api.env`). For security reasons, I've hidden my own API key via `.gitignore`.

In [23]:
# Load the .env file containing the OpenAI API Key
load_dotenv('api.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI()



sk-proj-3gBM6QrYwRB7PVwmZYceT3BlbkFJMyghsK5T5kBzJP34waqn
